In [31]:
from tensorflow import keras
shakespeare_url = "https://homl.info/shakespeare" # shortcut URL
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [32]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [33]:
tokenizer.texts_to_sequences("hi")
tokenizer.sequences_to_texts([[1, 2, 3, 4, 5]])

['  e t o a']

In [34]:
max_ids = len(tokenizer.word_index)
max_ids

39

In [35]:
dataset_size = tokenizer.document_count
dataset_size

1115394

In [36]:
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [37]:
import tensorflow as tf

train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [38]:
n_steps = 150
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [39]:
dataset = dataset.flat_map(lambda window:window.batch(window_length))

In [40]:
dataset = dataset.shuffle(10000)
dataset = dataset.map(lambda window: (window[:-1], window[1:]), num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)


In [41]:
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, LayerNormalization, Dense, Dropout, TimeDistributed
import os

def build_model(max_ids: int,
                embedding_dim: int = 128,
                gru_units: int = 256,
                dense_units: int = 128,
                dropout_rate: float = 0.2) -> keras.Model:
    """
    Define and return a Sequential model with:
      - Embedding
      - Two stacked bidirectional GRUs + layer norms
      - A Dense block + Dropout
      - TimeDistributed softmax output
    """
    return keras.models.Sequential([
        Embedding(input_dim=max_ids, output_dim=embedding_dim),
        Bidirectional(GRU(gru_units, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)),
        LayerNormalization(),
        Bidirectional(GRU(gru_units, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)),
        LayerNormalization(),
        Dense(dense_units, activation="relu"),
        Dropout(0.3),
        TimeDistributed(Dense(max_ids, activation="softmax"))
    ])


def compile_model(model: keras.Model,
                  initial_lr: float = 5e-4,
                  decay_steps: int = 5000,
                  decay_rate: float = 0.9,
                  staircase: bool = True) -> None:
    """
    Attach an Adam optimizer with an exponential decay LR schedule,
    compile with sparse_categorical_crossentropy + accuracy.
    """
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=initial_lr,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=staircase
    )
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=optimizer,
                  metrics=["accuracy"])


def train_model(model: keras.Model,
                dataset,
                epochs: int = 10,
                validation_data=None,
                checkpoint_path: str = "best_model.keras",
                patience: int = 5,
                verbose: int = 1) -> keras.callbacks.History:
    """
    Train given model on `dataset`.
      - Uses EarlyStopping on val_loss (if validation_data provided).
      - Saves best model to `checkpoint_path`.
      - Returns the training History object.
    """
    callbacks = []
    if validation_data is not None:
        callbacks += [
            keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_path,
                save_best_only=True,
                monitor="val_loss",
                verbose=verbose
            ),
            keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=patience,
                restore_best_weights=True,
                verbose=verbose
            )
        ]
    history = model.fit(
        dataset,
        epochs=epochs,
        validation_data=validation_data,
        callbacks=callbacks,
        verbose=verbose
    )
    return history
def evaluate_model(model: keras.Model,
                   dataset,
                   verbose: int = 1) -> dict:
   
    # Run evaluation
    results = model.evaluate(dataset, verbose=verbose)
    names = model.metrics_names  # e.g. ['loss', 'accuracy']
    metrics = dict(zip(names, results))

    # Print them nicely
    print("\nEvaluation results:")
    for name, value in metrics.items():
        print(f"  {name.capitalize():<10}: {value:.4f}")

    return metrics


def save_model(model: keras.Model, filepath: str) -> None:
    """Save the entire model to the given filepath."""
    dirname = os.path.dirname(filepath)
    if dirname and not os.path.exists(dirname):
        os.makedirs(dirname)
    #This is the main command to save the model
    model.save(filepath)
    print(f"Model saved to {filepath}")


def load_model(filepath: str) -> keras.Model:
    """Load and return a model from disk."""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"No model found at: {filepath}")
    print(f"Loading model from {filepath}")
    #This is the command to load the model
    return keras.models.load_model(filepath)




In [42]:
if(os.path.exists("best_shakespeare_model.keras")):  # Check if the model exists
    print("Loading existing model...")
    model = load_model("best_shakespeare_model.keras")  # Load the best model if it exists
else:
    print("Building new model...")
    model = build_model(max_ids=max_ids)

Loading existing model...
Loading model from best_shakespeare_model.keras


In [43]:
compile_model(model, initial_lr=5e-4, decay_steps=5000, decay_rate=0.9, staircase=True)
history = train_model(model, dataset=dataset, epochs=0, validation_data=None, checkpoint_path="best_shakespeare_model.keras", patience=5, verbose=1)


In [48]:
small_eval_dataset = dataset.take(100)
evaluate_model(model, small_eval_dataset, verbose=1)


100/100 ━━━━━━━━━━━━━━━━━━━━ 39s 358ms/step - accuracy: 0.9967 - loss: 0.0108

Evaluation results:
  Loss      : 0.0109
  Compile_metrics: 0.9967


{'loss': 0.010876116342842579, 'compile_metrics': 0.9967038631439209}

now lets just satrt to run the model for basic nlp tasks.Ths is how basic NLPS work they just they start predicting the next word formt eh given words that have been given to them.Also there are again used as input for further language preocessing and prediction

In [47]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(X,max_ids)
def predict_next_char(model: keras.Model, tokenizer, seed_text: str, max_sequence_len: int) -> str:
    padded_input = keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([seed_text.lower()]), maxlen=max_sequence_len, padding='pre')
    predicted_char_idx = np.argmax(model.predict(padded_input, verbose=0)[0, -1])
    return tokenizer.index_word.get(predicted_char_idx, "")
# Example usage
seed_text = "Hi"
predict_next_char(model, tokenizer, seed_text, n_steps)


'o'

In [ ]:
def